# I394E managed lanes.

In this notebook we will run through:

1. Using a configuration file to run lasso  
2. Setting up a base scenario and applying projects  
3. Transforming the standard network format to the MetCouncil expected format    
4. Exporting the network to a shapefile and csvs  

In [3]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit

In [4]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [5]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)

## Read a Config File

Let's examine the configuration file and store it as `my_config` variable.  

Configuration files are written in YAML and read in as python dictionaries.

In [7]:
MY_CONFIG_FILE = "C:/projects/met_council_git/project_cards/config_test.yml"

with open(MY_CONFIG_FILE) as f:
        my_config = yaml.safe_load(f)
        
import json
print(json.dumps(my_config, indent=2))

## Alternatively this could be written in the notebook our selected via a notebook GUI

{
  "base_scenario": {
    "input_dir": "C:/projects/met_council_git/data/interim/networks/base_network",
    "shape_file_name": "shape.geojson",
    "link_file_name": "link.json",
    "node_file_name": "node.geojson",
    "transit_directory": "",
    "validate_network": false,
    "lasso_base_dir": "D:/lasso"
  },
  "scenario": {
    "project_cards_filenames": [
      "C:/projects/met_council_git/project_cards/managed_lane_examples/hov_ramp_managed_lane.yml",
      "C:/projects/met_council_git/project_cards/managed_lane_examples/i394E_parallel_managed_lane.yml",
      "C:/projects/met_council_git/project_cards/managed_lane_examples/i394E_reversible_managed_lane.yml",
      "C:/projects/met_council_git/project_cards/year_2018_corrections_assign_group.yml",
      "C:/projects/met_council_git/project_cards/year_2018_corrections_assign_group_roadway_class.yml"
    ],
    "card_directory": [],
    "glob_search": [],
    "tags": []
  },
  "output": {
    "format": "geojson",
    "directory"

## Create a Base Scenario

Base scenarios must at the least specify a highway network but can also specify a directory where transit networks can be found.  

In this step the highway and transit networks are read in and validated to each other.  

In some cases, you may want to override the validation (after reviewing the errors) using the flag: `validate = False`.

In [8]:
base_wrangler_path = os.path.join(os.path.dirname((os.path.dirname(os.path.abspath('')))),"network_wrangler")
WranglerLogger.info("Base Wrangler Path: {}".format(base_wrangler_path))

base_scenario = Scenario.create_base_scenario(
        my_config["base_scenario"]["shape_file_name"],
        my_config["base_scenario"]["link_file_name"],
        my_config["base_scenario"]["node_file_name"],
        roadway_dir=my_config["base_scenario"]["input_dir"],
        transit_dir=my_config["base_scenario"]["input_dir"],
        validate = False,
    
    )

2020-07-28 14:24:22, INFO: Base Wrangler Path: D:\network_wrangler
2020-07-28 14:24:22, INFO: Base Wrangler Path: D:\network_wrangler
2020-07-28 14:24:22, INFO: Reading from following files:
-C:/projects/met_council_git/data/interim/networks/base_network\link.json
-C:/projects/met_council_git/data/interim/networks/base_network\node.geojson
-C:/projects/met_council_git/data/interim/networks/base_network\shape.geojson.
2020-07-28 14:24:22, INFO: Reading from following files:
-C:/projects/met_council_git/data/interim/networks/base_network\link.json
-C:/projects/met_council_git/data/interim/networks/base_network\node.geojson
-C:/projects/met_council_git/data/interim/networks/base_network\shape.geojson.
2020-07-28 14:29:17, INFO: Read 1134074 links from C:/projects/met_council_git/data/interim/networks/base_network\link.json
2020-07-28 14:29:17, INFO: Read 1134074 links from C:/projects/met_council_git/data/interim/networks/base_network\link.json
2020-07-28 14:29:17, INFO: Read 354346 nodes

2020-07-28 14:37:23, DEBUG: ...calendar_dates.txt:
Empty DataFrame
Columns: [service_id, date, exception_type]
Index: []
2020-07-28 14:37:23, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-07-28 14:37:23, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-07-28 14:37:24, DEBUG: ...fare_attributes.txt:
Empty DataFrame
Columns: [fare_id, price, currency_type, payment_method, transfers]
Index: []
2020-07-28 14:37:24, INFO: Removing fare_attributes.txt from transit network config because file not found
2020-07-28 14:37:24, INFO: Removing fare_attributes.txt from transit network config because file not found
2020-07-28 14:37:24, DEBUG: ...fare_rules.txt:
Empty DataFrame
Columns: [fare_id]
Index: []
2020-07-28 14:37:24, INFO: Removing fare_rules.txt from transit network config because file not found
2020-07-28 14:37:24, INFO: Removing fare_rules.txt from transit network config because file not found
2020-07-28 

2020-07-28 14:37:24, INFO: Removing transfers.txt from transit network config because file not found
2020-07-28 14:37:24, DEBUG: ...feed_info.txt:
Empty DataFrame
Columns: [feed_publisher_name, feed_publisher_url, feed_lang]
Index: []
2020-07-28 14:37:24, INFO: Removing feed_info.txt from transit network config because file not found
2020-07-28 14:37:24, INFO: Removing feed_info.txt from transit network config because file not found
2020-07-28 14:37:24, WARNING: The following stop_ids from are referenced but missing from stops.txt: ['place_BHPR', 'place_157G', 'place_BTST', 'place_CE47', 'place_EATS', 'place_PHPR', 'place_DFME', 'place_AVTS', 'place_14CE']
2020-07-28 14:37:24, WARNING: The following stop_ids from are referenced but missing from stops.txt: ['place_BHPR', 'place_157G', 'place_BTST', 'place_CE47', 'place_EATS', 'place_PHPR', 'place_DFME', 'place_AVTS', 'place_14CE']
2020-07-28 14:37:24, DEBUG: starting ox_graph()
2020-07-28 14:37:25, DEBUG: GRAPH NODES: Index(['model_node

#### Create project cards from projects that are explicitely specified in config


In [9]:
if len(my_config["scenario"]["project_cards_filenames"]) > 0:
    project_cards_list = [
        ProjectCard.read(filename, validate=False)
        for filename in my_config["scenario"]["project_cards_filenames"]
    ]
else: 
    project_cards_list = []
project_cards_list


## Create Scenario

A scenario is constructed with a base scenario and then selecting project cards to be added to that base scenario to create the new scenario.

Projects can be added a variety of ways:

 1. `card_directory` + `tags` will search a directory add project's who's project tags match *at least one of* the tags in the keyword.
 2. `card_directory` + `glob_search` will search a directory add project's who's file name matches the [glob search text](https://docs.python.org/3/library/glob.html)
 3. `project_cards_list` is a list of ProjectCard objects
 
Optionally, you may specify that project card formats are not validated by setting they keyword: 
   `validate = False`
   
Projects that are not added in the initial scenario development can be added by using the following methods:  

 - `add_project_card_from_file()`  
 - `add_project_cards_from_directory()`  
 - `add_project_cards_from_tags`
 
Or by directly adding the project to the scenario's project attribute by running:

```python
my_project = ProjectCard.read(path_to_card)
my_scenario.projects += my_project

```


In [10]:
my_scenario=None

my_scenario = Scenario.create_scenario(
    base_scenario=base_scenario,
    project_cards_list=project_cards_list,
    validate_project_cards=False,
)

2020-07-28 14:44:04, INFO: Creating Scenario
2020-07-28 14:44:04, INFO: Creating Scenario
2020-07-28 14:44:04, DEBUG: Adding project cards from List.
HOV ramp refactor,I394 Parallel Lanes,I394E Reversible Lanes,Correct Year 2018 Assignment Group,Correct Year 2018 Assignment Group and Roadway Class


### Apply all projects in scenario

In [11]:
WranglerLogger.info("\nProjects in queue to be applied: \n - {}".format("\n - ".join(my_scenario.get_project_names())))
WranglerLogger.info("\n[Before] Applied Projects: \n - {}".format("\n - ".join(my_scenario.applied_projects)))

my_scenario.apply_all_projects()

WranglerLogger.info("\n[After] Applied Projects: \n - {}".format("\n - ".join(my_scenario.applied_projects)))

2020-07-28 14:47:43, INFO: 
Projects in queue to be applied: 
 - HOV ramp refactor
 - I394 Parallel Lanes
 - I394E Reversible Lanes
 - Correct Year 2018 Assignment Group
 - Correct Year 2018 Assignment Group and Roadway Class
2020-07-28 14:47:43, INFO: 
Projects in queue to be applied: 
 - HOV ramp refactor
 - I394 Parallel Lanes
 - I394E Reversible Lanes
 - Correct Year 2018 Assignment Group
 - Correct Year 2018 Assignment Group and Roadway Class
2020-07-28 14:47:43, INFO: 
[Before] Applied Projects: 
 - 
2020-07-28 14:47:43, INFO: 
[Before] Applied Projects: 
 - 
2020-07-28 14:47:43, DEBUG: Ordered Project Cards: {'correct year 2018 assignment group and roadway class': <network_wrangler.projectcard.ProjectCard object at 0x00000224E2E28D08>, 'correct year 2018 assignment group': <network_wrangler.projectcard.ProjectCard object at 0x000002257E78C308>, 'i394e reversible lanes': <network_wrangler.projectcard.ProjectCard object at 0x00000224E0F3D808>, 'i394 parallel lanes': <network_wrang

d:\network_wrangler\network_wrangler\roadwaynetwork.py:862: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidate_links["i"] = 0


2020-07-28 14:47:43, INFO: Applying Correct Year 2018 Assignment Group
2020-07-28 14:47:43, INFO: Applying Correct Year 2018 Assignment Group
2020-07-28 14:47:43, INFO: Applying Correct Year 2018 Assignment Group
2020-07-28 14:47:43, INFO: Applying Correct Year 2018 Assignment Group
2020-07-28 14:47:43, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-07-28 14:47:43, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-07-28 14:47:43, DEBUG: validating selection
2020-07-28 14:47:43, DEBUG: Selection Key: ((model_link_id==381080 or model_link_id==381082 or model_link_id==381121 or model_link_id==381127 or model_link_id==381129 or model_link_id==381137 or model_link_id==381138 or model_link_id==381145 or model_link_id==381168 or model_link_id==381169 or model_link_id==381171 or model_link_id==381172 or model_link_id==381220 or model_link_id==381298 or model_link_id==381310 or model_link_id==381312 or model_link_id==381313 or

2020-07-28 14:47:43, DEBUG: Building selection query
2020-07-28 14:47:43, DEBUG: Selecting features:
((model_link_id==381080 or model_link_id==381082 or model_link_id==381121 or model_link_id==381127 or model_link_id==381129 or model_link_id==381137 or model_link_id==381138 or model_link_id==381145 or model_link_id==381168 or model_link_id==381169 or model_link_id==381171 or model_link_id==381172 or model_link_id==381220 or model_link_id==381298 or model_link_id==381310 or model_link_id==381312 or model_link_id==381313 or model_link_id==381314 or model_link_id==381318 or model_link_id==381322 or model_link_id==381330 or model_link_id==381341 or model_link_id==381347 or model_link_id==381348 or model_link_id==381354 or model_link_id==381389 or model_link_id==381390 or model_link_id==381417 or model_link_id==381455 or model_link_id==381481 or model_link_id==381489 or model_link_id==381501 or model_link_id==381504 or model_link_id==381509 or model_link_id==381511 or model_link_id==381670 

SystemError: <built-in function is_scalar> returned a result with an error set

In [ ]:
#my_scenario.road_net.links_df.info()

In [ ]:
#my_scenario.road_net.links_df[my_scenario.road_net.links_df["segment_id"] == 5]

# Write out as MetCouncil Model Roadway Network
Everything above was done in "pure wrangler" rather than lasso.  However, we will need Lasso in order to add the MetCouncil specific variables. You can create a lasso ModelRoadwayNetwork object from the roadway network object and feed it any additional parameters from that `my_config` variable.

You can see that the link variables for this network are the same as the standard roadway network at this point but that will change.

Since this is a GeoDataFrame you can also use build-in Geopandas features to make simple plots based on these variables.

In [ ]:
my_config

In [ ]:
model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    my_scenario.road_net, parameters=my_config.get("my_parameters", {})
)

In [ ]:
#model_road_net.links_df.info()

In [ ]:
#model_road_net.links_df.loc[model_road_net.links_df["HOV"] == "NaN", "HOV"] = "0"

In [ ]:
#model_road_net.links_df[model_road_net.links_df["HOV"] != "0"]

In [ ]:
#model_road_net.nodes_df.info()

## Add MetCouncil variables
At this point, we need to calculate all the variables into what MetCouncil's model is expecting. The method `roadway_standard_to_met_council_network()` broadly does the following:  
 
- creates a parallel managed lane network
- calculates additional variables based on geography or other variables (i.e. county, assignment group, area type, etc)
- flattens variables stored as continuous time values and determines their value by time period (i.e. lanes_am)   
- reprojects into MetCouncil's projection

In [ ]:
model_road_net.roadway_standard_to_met_council_network()

In [ ]:
model_road_net.links_df[model_road_net.links_df.assign_group==""]

In [ ]:
model_road_net.links_df.assign_group.unique()

In [ ]:
#model_road_net.links_metcouncil_df.query('lanes == 0 & drive_access == 1 & centroidconnect')

In [ ]:
#pd.options.display.max_columns = None
#model_road_net.links_metcouncil_df[model_road_net.links_metcouncil_df["model_link_id"] == 1390975]["segment_id"]


In [ ]:
#model_road_net.links_metcouncil_df[model_road_net.links_metcouncil_df.model_link_id.isin([390975])]

In [ ]:
#model_road_net.links_metcouncil_df.segment_id.value_counts()
pd.crosstab(model_road_net.links_metcouncil_df.segment_id, model_road_net.links_metcouncil_df.managed)

In [ ]:
#model_road_net.links_metcouncil_df.plot("lanes_AM")

## Export to shapefile

As a last step, the network can be exported to a shapefile and paired CSVs after removing extraneous variables.

(note that this step will also run the `roadway_standard_to_met_council_network()` method but I wanted to show it to you piecewise)

In [ ]:
model_road_net.write_roadway_as_shp(
    output_link_shp = "C:/projects/met_council_git/data/interim/networks/fixed_width/link.shp",
    output_node_shp = "C:/projects/met_council_git/data/interim/networks/fixed_width/node.shp"
)

# Export to fixed width file

In [ ]:
model_road_net.write_roadway_as_fixedwidth(
    output_link_txt = "C:/projects/met_council_git/data/interim/networks/fixed_width/link.txt",
    output_node_txt = "C:/projects/met_council_git/data/interim/networks/fixed_width/node.txt",
    output_link_header_width_txt = "C:/projects/met_council_git/data/interim/networks/fixed_width/link_header.csv",
    output_node_header_width_txt = "C:/projects/met_council_git/data/interim/networks/fixed_width/node_header.csv",
    output_cube_network_script = "C:/projects/met_council_git/data/interim/networks/fixed_width/make_complete_network_from_fixed_width_file.s"
)

# Write out as MetCouncil Model Transit Network

Similar to the roadway network, the first step is to convert it to a Lasso object, and then write it to a cube line file.  Optionally, you could also export it to a shapefile to inspect using other means. 

In [27]:
#standard_transit = StandardTransit.fromTransitNetwork(my_scenario.transit_net)
#standard_transit.feed

Write out the StandardTransit Lasso object to a cube line file:

In [28]:
#standard_transit.write_as_cube_lin()

In [29]:
model_road_net.links_metcouncil_df[model_road_net.links_metcouncil_df.model_link_id == 400170].assign_group

400327    2
Name: assign_group, dtype: int32

In [30]:
my_scenario.road_net.links_df[my_scenario.road_net.links_df.model_link_id == 400170].assign_group

400169    2.0
Name: assign_group, dtype: float64